In [1]:
import numpy as np # linear algebra
import pandas as pd
from random import uniform

# Function to generate random data which does not change the mean and std of the distribution

In [2]:
def getUniformData(df,colname):
    Column_Name_avg = df[colname].mean()
    Column_Name_std = df[colname].std()
    Column_Name_null_count = df[colname].isnull().sum()
    Column_Name_null_random_list = Column_Name_std * uniform(0,1) + Column_Name_avg
    df[colname][np.isnan(df[colname])] = Column_Name_null_random_list
    return df,Column_Name_null_random_list

# Fill the NAN values of the dataframe (numerical)

In [3]:
def FillNANinNumericalColumns(df):
    df,Column_Name_null_random_list_customer=getUniformData(df,'customer_score')
    df,Column_Name_null_random_list_month=getUniformData(df,'months_of_activity')
    c_score_mean = df['anon_var_1'].mean() # the nan values will be replaced by the mean
    df['anon_var_1'].fillna(value=c_score_mean,inplace=True)
    return Column_Name_null_random_list_customer,Column_Name_null_random_list_month,c_score_mean,df

# Frequency encoding of the categorical columns of the dataframe

In [4]:
def FrequencyEncodingCategoricalColumn(df):
    cols = df.columns
    num_cols = df._get_numeric_data().columns
    categorical_col= list(set(cols) - set(num_cols))
    for c in (categorical_col):
        df[c+'_freq'] = df[c].map(df.groupby(c).size() / df.shape[0])
        df= df.drop(c,axis=1)
    return df

In [5]:
df= pd.read_csv('train.csv')

# remove the sex and id columns before processing

In [6]:
df.drop(['sex','id'],axis=1,inplace=True)

# get the list of values used to fill the NAN values of the columns for writing the same to test dataframe

In [7]:
Column_Name_null_random_list_customer,Column_Name_null_random_list_month,c_score_mean,df = FillNANinNumericalColumns(df) 

/work/test/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


# fill the categorical columns NAN with Z so that it can later be replaced with the accuracy

In [8]:
df['customer_score_confidence'] = df['customer_score_confidence'].fillna('Z')
df['taxi_type'] = df['taxi_type'].fillna('Z')
df = FrequencyEncodingCategoricalColumn(df)

# Get training labels

In [9]:
train_labels = df['pricing_category']
train_labels = train_labels.values
train_labels= train_labels.astype('float64')
df= df.drop(['pricing_category'],axis=1)

In [10]:
train_labels

array([2., 2., 2., ..., 2., 2., 3.])

In [11]:
train_features = df.values

In [12]:
from sklearn.ensemble import GradientBoostingClassifier
clf = GradientBoostingClassifier(learning_rate=0.1, min_samples_split=500,min_samples_leaf=50,max_depth=8,max_features='sqrt',subsample=0.8,random_state=10)

In [13]:
clf.fit(train_features,train_labels)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=8,
              max_features='sqrt', max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=50, min_samples_split=500,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              n_iter_no_change=None, presort='auto', random_state=10,
              subsample=0.8, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False)

In [14]:
test_df = pd.read_csv('test.csv')

In [15]:
df_test= test_df.drop(['id','sex'],axis=1)

# perform the same preprocessing on the NAN values of the columns

In [16]:
df_test['customer_score'][np.isnan(df_test['customer_score'])] = Column_Name_null_random_list_customer
df_test['months_of_activity'][np.isnan(df_test['months_of_activity'])] = Column_Name_null_random_list_month
df_test['anon_var_1'].fillna(value=c_score_mean,inplace=True)

/work/test/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/work/test/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [17]:
df_test['customer_score_confidence'] = df_test['customer_score_confidence'].fillna('Z')
df_test['taxi_type'] = df_test['taxi_type'].fillna('Z')
df_test = FrequencyEncodingCategoricalColumn(df_test)

In [18]:
test_features= df_test.values

In [19]:
test_labels= clf.predict(test_features)

# saves the result label and id to the submission.csv file

In [20]:
test_df["pricing_category"]=test_labels
test_df[["id","pricing_category"]].to_csv("submission.csv",index=False)